In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
from ROOT import TChain

In [ ]:
def open_df(fname,treename="flashtrigger/opflash_opflash_tree",start=None,stop=None):
    return pd.DataFrame( root2array(fname, treename,start=start,stop=stop) )

In [ ]:
#
# Data loading
#
rate=5
minutes=0
hour=0
day=1

#num_cosmic = 5*60*(minutes+hour*60+day*60*24)
#num_bnb = int(num_cosmic/670)+1

#print 'need',num_cosmic,'cosmic'
#print 'need',num_bnb,'bnb'

df_mc_cosmic = open_df('flash_trigger_ana/cosmic.root')
df_data_cosmic = open_df('flash_trigger_ana/data_cosmic.root')
#df_bnb = open_df('flash_trigger_ana/bnb.root')



In [ ]:
#plt.ylim(ymin,ymax)
#plt.xlim(-400,400)
#ax.set_yscale('log')
dt_data=df_data_cosmic.query('pe_total>50')['dt'].ravel()
print dt_data.min(),dt_data.max()
yval,xbins = np.histogram(dt_data,bins=3,range=(-1600,3200))

nevents=len(df_data_cosmic.groupby(['run','subrun','event']))
ctr_per_frame=[]
ctr_per_frame_err=[]
for x in yval:
    ctr_per_frame.append(float(x) / nevents)
    ctr_per_frame_err.append(np.sqrt(x)/nevents)
    
plt.errorbar(np.arange(0,3,1),ctr_per_frame,yerr=ctr_per_frame_err,marker='o', linestyle='--',color='blue')
plt.xlim(-.5,2.5)
plt.ylim(0,10)
plt.xlabel('Frame Number',fontsize=20)
plt.ylabel('OpFlash Counts',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('Data OpFlash (> 50 P.E.) Counts per Frame',fontsize=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
#plt.ylim(ymin,ymax)
#plt.xlim(-400,400)
#ax.set_yscale('log')
drift_window_df=df_data_cosmic.query('pe_total>50 and -1600 < dt and dt < 3200')

flash_count=[]
for key,df in drift_window_df.groupby(['run','subrun','event']):
    flash_count.append(len(df))
flash_count=np.array(flash_count)




In [ ]:
drift_data_window_df=df_data_cosmic.query('pe_total>50 and 100 < dt and dt < 2100')

flash_data_count=[]
for key,df in drift_data_window_df.groupby(['run','subrun','event']):
    flash_data_count.append(len(df))
flash_data_count=np.array(flash_data_count)

drift_mc_window_df=df_mc_cosmic.query('pe_total>50 and 100 < dt and dt < 2100')

nevents_data = len(df_data_cosmic.groupby(['run','subrun','event']))
nevents_mc   = len(df_mc_cosmic.groupby(['run','subrun','event']))

flash_mc_count=[]
for key,df in drift_mc_window_df.groupby(['run','subrun','event']):
    flash_mc_count.append(len(df))
flash_mc_count=np.array(flash_mc_count)

mc_weights = [float(nevents_data) / float(nevents_mc)] * len(flash_mc_count)

fig, ax = plt.subplots(figsize=(12, 8))
plt.hist(flash_data_count,
        bins=100,range=(0,100),
        label='Data Cosmic (Run 2377)',alpha=0.5,color='magenta')
plt.hist(flash_mc_count,
        bins=100,range=(0,100),weights=mc_weights,
        label='MCC 6.1 Cosmic',alpha=0.5,color='blue')
plt.ylim(0,8000)
plt.xlim(0,30)
plt.xlabel('OpFlash Count per Drift Window (2ms)',fontsize=20)
plt.ylabel('OpFlash Counts',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('Data OpFlash (> 50 P.E.) Counts per Drift Window (2ms)',fontsize=20)
ax.legend(prop={'size':20})
plt.show()